In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd drive/MyDrive/Colab\ Notebooks


In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
#import random
from tqdm.notebook import tqdm
from dyn_env7 import EngagementDynamics
import pickle
from torch import optim

from sklearn.preprocessing import MinMaxScaler
from easydict import EasyDict as edict
from pngdataset import TrainTestDataSeperator, state2latent, scaled_data, RolloutSequenceDataset, RelationNetworkDataset

### Parameter Settings

In [4]:
args = edict({
    "scaler_s": MinMaxScaler(),
    "scaler_a": MinMaxScaler(),
    "scaler_d": MinMaxScaler(),
    "latents": 6,
    "actions": 1,
    "hiddens": 126,
    "g_hiddens": 80,
    "seq_len": 100,
    "batch_size": 500,
    "rn_batch_size":30,
    "learning_rate": 0.0001,
    "rn_learning_rate":0.0003,
    "lstm_epochs": 100,
    "rn_epochs": 500,
    "device": 'cuda',
    "quantile": [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    "class_num": 4
})

## 1. Training, validation and  test data seperation
- pickle 파일로 저장
- dictionary keys : states, actions, missD, index

In [5]:
with open('./dataset/unspecified/unspecified.pickle', 'rb') as f:
    Dataset = pickle.load(f)

TrainTestDataSeperator(Dataset,train = 60, validation = 0)

Total Datasets : 320
------------------------------------------------------------------------------
train : 192 
validation : 0 
test : 128
Data index 
 [2, 3, 7, 9, 13, 14, 16, 17, 18, 23, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 37, 38, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 57, 58, 59, 60, 62, 64, 66, 67, 68, 69, 70, 74, 77, 78] 
 [] 
 [0, 1, 4, 5, 6, 8, 10, 11, 12, 15, 19, 20, 21, 22, 26, 31, 36, 39, 40, 41, 49, 54, 56, 61, 63, 65, 71, 72, 73, 75, 76, 79]


## 2. Data preprocessing

- 학습 데이터 전처리는 다음의 3 파트로 구성
 1. Selecting latent vector
 2. Data fitting with scaler
 3. Seperation by sequence length


1. Selecting latent vector
 - 기존의 state 데이터를 학습이 더 잘 되도록 변경
 - state vector  :  $[Xm, Ym, Vx, Vy, \gamma]$
 - latent vector :  $[Rx, Ry, Vrx, Vry, \cos(\gamma), \sin(\gamma)]$


2. Data fitting with scaler
 - 데이터 간격을 일정하게 하기 위해 scaler 도입
   - Scaler의 종류
     1. MinMaxScaler(feature_range = (0,1)) : 최소값 0, 최대값 1이 되도록 scaling
     2. StandardScaler() : 평균이 0, 분산이 1인 가우시안으로 scaling
 - scaler 간격 [0,1]
 - 데이터 저장 경로: ./dataset/scaled_factor

 
3. Seperation by seqeunce length
 - LSTM에 들어가기 위해 time series data를 sequence length 만큼 잘라냄
 - Sequence length:

In [6]:
# train, test data 불러오기
with open('./dataset/train/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('./dataset/test/test.pickle', 'rb') as f:
    test = pickle.load(f)

# state-> latent data 로 변경
train_latent = state2latent(train)
test_latent = state2latent(test)

# Data fitting으로 scaler 사용
train_scaled = scaled_data(train_latent, args, train = True)
test_scaled = scaled_data(test_latent, args, train = False)

# seperation by sequence length & Dataloader
#transform true는 numpy -> torch해줄거니?
train_dataset = RolloutSequenceDataset(train_scaled, args, transform = True)
test_dataset = RolloutSequenceDataset(test_scaled, args, transform = True)

train_loader = DataLoader(train_dataset, batch_size = args.batch_size, num_workers = 2, 
                          shuffle = True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size = args.batch_size, num_workers = 2,
                          shuffle = False, drop_last = True)

## 3. Train LSTM

- LSTM 학습은 다음의 3 파트로 구성
 1. LSTM 모델 구성
 2. Train LSTM
 3. Plotting Average Loss


In [7]:
class LSTM(nn.Module):
    def __init__(self, args):
        super(LSTM, self).__init__()
        self.latents = args.latents
        self.actions = args.actions
        self.hiddens = args.hiddens
        self.seq_len = args.seq_len
        self.device = args.device
        self.lstm = nn.LSTMCell(self.latents + self.actions, self.hiddens)
        self.mlp = nn.Linear(self.hiddens, self.latents)
         
    def forward(self, latents, actions):
        bs, seq_len = actions.size(0), actions.size(1)
        device = self.device
        ins = torch.cat([latents, actions], dim = -1)
        # initialize hidden and cell states
        # (batch, hidden_size)
        h0 = torch.zeros(bs, self.hiddens).to(device)
        c0 = torch.zeros(bs, self.hiddens).to(device)
        
        h_t = h0
        c_t = c0
        preds = torch.Tensor([]).to(device)

        # ht: (batch_size, hidden_size)
        # hts: (batch_size, seq_length, hidden_size)
        #out : (batch_size, latent_size)
        #outs: (batch_size, latent_size , seq_length)
        for i in range(seq_len):

            x = ins[:,i]
            h_t,c_t = self.lstm(x,(h_t,c_t))
            pred = self.mlp(h_t)
            preds = torch.cat((preds, pred), dim =-1)
        
        preds = preds.view(-1, self.seq_len, self.latents)

        return preds, h_t

In [8]:
"""
Model loading
"""
model = LSTM(args)
criterion = nn.MSELoss()

optimizer = optim.Adam(params=model.parameters(), lr = args.learning_rate)
device = 'cuda'

In [9]:
"""
Training LSTM
"""
model.train()

for epoch in range(args.lstm_epochs):
    losses = []
    test_losses = []
    
    if epoch%5 == 0:
        pbar = tqdm(total=len(train_loader), desc = "EPOCH {}".format(epoch))
    for it, (latent, action, next_latent, missD, index) in enumerate(train_loader):
        # For GPU processing
        latent = latent.to(device)
        next_latent = next_latent.to(device)
        action = action.to(device)
        model = model.to(device)
        
        pred, h_t = model(latent,action)
        
        optimizer.zero_grad()
        loss = criterion(pred, next_latent)
        losses.append(loss.item())

        
        if epoch%5 ==0:
            pbar.set_postfix_str("Loss={curr_loss:10.6f}, Average loss={avg_loss:10.6f}".format(curr_loss=loss.item()*100,avg_loss=sum(losses)/len(losses)*100))
            pbar.update(1)
            
        loss.backward()
        optimizer.step()
        
    if epoch%5 == 0:
        pbar.close()

EPOCH 0:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 5:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 10:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 15:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 20:   0%|          | 0/648 [00:00<?, ?it/s]

...

EPOCH 75:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 80:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 85:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 90:   0%|          | 0/648 [00:00<?, ?it/s]

EPOCH 95:   0%|          | 0/648 [00:00<?, ?it/s]

## 4. Train Relational Learning Model

- Relational Learning model은 다음의 3 파트로 구성
 1. h 추출 모델
 2. time series, non-time series 합치는 모델
 3. relation 학습 모델
 
 

In [10]:
train_dataset = RelationNetworkDataset(train_scaled, args, transform = True)
test_dataset = RelationNetworkDataset(test_scaled, args, transform = True)

train_loader = DataLoader(train_dataset, batch_size = args.rn_batch_size , num_workers = 2, 
                          shuffle = True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size = 32 , num_workers = 2,
                          shuffle = True, drop_last = True)

100%|██████████| 128/128 [02:02<00:00,  1.05it/s]


In [11]:
class RN(nn.Module):
    def __init__(self, args):
        super(RN, self).__init__()
        self.actions = args.actions
        self.hiddens = args.hiddens
        self.latents = args.latents
        self.g_hiddens = args.g_hiddens
        self.quantile_num = len(args.quantile)
        self.class_num = args.class_num
        self.g1 = nn.Linear(2*(args.latents +args.actions + args.hiddens + 1), 128)
        self.g2 = nn.Linear(128, 64)
        self.g3  = nn.Linear(64, 32)
        self.f1 = nn.Linear(32, 16)
        self.f2 = nn.Linear(16,8)
        self.f3 = nn.Linear(8, self.class_num)
        self.act1 = nn.LeakyReLU(0.01)
        self.act2 = nn.LeakyReLU(0.01)
        self.act3 = nn.LeakyReLU(0.01)
        self.act4 = nn.LeakyReLU(0.01)

        
    def forward(self,latent_pair, action_pair, hidden_pair, missD_pair):
        # 후보로는 los_rate 넣기
        # 1. class 별로 object pair 나눠주는애
        # 2. h 뽑아주는애(time step, quantile로)
        (l1, l2) = latent_pair
        (a1, a2) = action_pair 
        (h1, h2) = hidden_pair #(bs, 6, 126)
        (missD1, missD2) = missD_pair

        l1 = l1.to(args.device)
        l2 = l2.to(args.device)
        a1 = a1.to(args.device)
        a2 = a2.to(args.device)
        h1 = h1.to(args.device)
        h2 = h2.to(args.device)
        missD1 = missD1.to(args.device)
        missD2 = missD2.to(args.device)

        bs = a1.size(0)
        
        outs = torch.Tensor([]).to(args.device)
        o1 = torch.cat((l1, a1, h1), dim = -1) 
        o2 = torch.cat((l2, a2, h2), dim = -1) 
        o1 = o1.to(args.device)
        o2 = o2.to(args.device)
        for i in range(self.quantile_num):
            
            object_pair = torch.cat((o1[:,i], o2[:,i], missD1, missD2), dim = -1)
            out1 = self.g1(object_pair)
            out2 = self.g2(self.act1(out1))
            out3 = self.g3(self.act2(out2))
            outs = torch.cat((outs, out3), dim =-1)
            
        outs = outs.view(bs, self.quantile_num, -1)
        
            
            
        # 3. outs에 대한 elementwise sum 해주는 애
        #outs_ews = torch.sum(outs, 1) #(bs, 2*50)
        outs_ews_md = torch.sum(outs, 1)
        
        pred1 = self.f1(outs_ews_md)
        pred2 = self.f2(self.act3(pred1))
        pred = self.f3(self.act4(pred2))
        return pred

In [12]:
"""
Model loading
"""
rn = RN(args)
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(params=rn.parameters(), lr = args.rn_learning_rate)
device = 'cuda'
args.device = 'cuda'
lstm = LSTM(args)
# lstm model loading
lstm.load_state_dict(torch.load('./saved_model/cur_lstm.pth'))
lstm = lstm.to(args.device)
rn = rn.to(args.device)
rn.train()
best_accuracy = -10
lstm.eval()

LSTM(
  (lstm): LSTMCell(7, 126)
  (mlp): Linear(in_features=126, out_features=6, bias=True)
)

In [13]:
test_loader

In [14]:
# 학습이 되기는 되는데 오버 피팅 되는 느낌이 있음(epoch 30 이상부터 오버피팅- > )
# f = 2, g = 3이였을 때 최대 epoch 35에서 65
def test(model, test_loader, args):
    correct = [0, 0, 0, 0] # normal, autopilot lag, LOS rate delay, LOS rate bias
    total = [0, 0, 0, 0]
    model.eval()
    model = model.to(args.device)
    with torch.no_grad():

        for it, (latent_pair, action_pair, missD_pair, index) in enumerate(test_loader):
            index = index.to(args.device)

          # lstm에서 뽑은 h 값 대입 - action값 재정립 필요 (100개인거 1 개로 바꿔야함)
            (l1, l2) = latent_pair
            (a1, a2) = action_pair
            (missD1, missD2) = missD_pair
            # for GPU processing
            l1 = l1.to(args.device)
            l2 = l2.to(args.device)
            a1 = a1.to(args.device)
            a2 = a2.to(args.device)
            missD1 = missD1.to(args.device)
            missD2 = missD2.to(args.device)
            index = index.to(args.device)

            (l1, l2) = latent_pair
            (a1, a2) = action_pair
            (missD1, missD2) = missD_pair

            h1 = torch.Tensor([]).to(args.device) #(bs, 6, 126)
            h2 = torch.Tensor([]).to(args.device)
            bs = a1.size(0)
            #action shape (bs, 6, 100, 1) -> 6* (bs, 100,1)
            for q in range(len(args.quantile)):
                l1_q = l1[:,q].to(args.device)
                l2_q = l2[:,q].to(args.device)
                a1_q = a1[:,q].to(args.device)
                a2_q = a2[:,q].to(args.device)

                _, h1_q = lstm(l1_q, a1_q)
                _, h2_q = lstm(l2_q, a2_q)
                h1 = torch.cat((h1, h1_q.unsqueeze(0)), dim = 0)
                h2 = torch.cat((h2, h2_q.unsqueeze(0)), dim = 0)
            h1 = h1.view(bs, len(args.quantile),-1)
            h2 = h2.view(bs, len(args.quantile),-1)
            hidden_pair = (h1, h2) #(30, 6, 126)

            action_pair = (a1[:, :, -1], a2[:, :, -1]) #(30, 6, 1)
            latent_pair = (l1[:, :, -1], l2[:, :, -1])
            pred = model(latent_pair, action_pair, hidden_pair, missD_pair)
            _, predicted = torch.max(pred, 1)
            #total += index.size(0)
            #correct += (predicted == index.long().squeeze()).sum().item()

            for i in range(len(index)):
                index_actual = int(index[i])
                index_predicted = predicted[i]
                total[index_actual] +=1
                if index_actual == index_predicted:
                    correct[index_actual] +=1
          
        accuracy = [0,0,0,0]
        for i in range(args.class_num):
            accuracy[i] = 100 *correct[i]/total[i]
        total_accuracy = 100* sum(correct) /sum(total)

    return total_accuracy, accuracy[0], accuracy[1], accuracy[2], accuracy[3]

In [15]:
"""
Training RN
"""
rn.train()
for epoch in range(args.rn_epochs):
    losses = []
    test_losses = []
    
    if epoch%5 == 0:
        pbar = tqdm(total=len(train_loader), desc = "EPOCH {}".format(epoch))
    for it, (latent_pair, action_pair, missD_pair, index) in enumerate(train_loader):
        # lstm에서 뽑은 h 값 대입 - action값 재정립 필요 (100개인거 1 개로 바꿔야함)
        (l1, l2) = latent_pair
        (a1, a2) = action_pair
        (missD1, missD2) = missD_pair
        # for GPU processing
        l1 = l1.to(args.device)
        l2 = l2.to(args.device)
        a1 = a1.to(args.device)
        a2 = a2.to(args.device)
        missD1 = missD1.to(args.device)
        missD2 = missD2.to(args.device)
        index = index.to(args.device)

        (l1, l2) = latent_pair
        (a1, a2) = action_pair
        (missD1, missD2) = missD_pair

        h1 = torch.Tensor([]).to(args.device) #(bs, 6, 126)
        h2 = torch.Tensor([]).to(args.device)
        #action shape (bs, 6, 100, 1) -> 6* (bs, 100,1)
        for q in range(len(args.quantile)):
            l1_q = l1[:,q].to(args.device)
            l2_q = l2[:,q].to(args.device)
            a1_q = a1[:,q].to(args.device)
            a2_q = a2[:,q].to(args.device)
            _, h1_q = lstm(l1_q, a1_q)
            _, h2_q = lstm(l2_q, a2_q)
            h1 = torch.cat((h1, h1_q.unsqueeze(0)), dim = 0)
            h2 = torch.cat((h2, h2_q.unsqueeze(0)), dim = 0)
        h1 = h1.view(args.rn_batch_size, len(args.quantile),-1)
        h2 = h2.view(args.rn_batch_size, len(args.quantile),-1)
        hidden_pair = (h1, h2) #(30, 6, 126)
        
        latent_pair = (l1[:, :, -1], l2[:, :, -1])
        action_pair = (a1[:, :, -1], a2[:, :, -1]) #(30, 6, 1)
        pred = rn(latent_pair, action_pair, hidden_pair, missD_pair)

        optimizer.zero_grad()
        loss = criterion(pred,index.squeeze().long())#criterion(pred, index_one_hot.long())
        losses.append(loss.item())

        if epoch%5 ==0:
            pbar.set_postfix_str("Loss={curr_loss:10.6f}, Average loss={avg_loss:10.6f}".format(curr_loss=loss.item(),avg_loss=sum(losses)/len(losses)))
            pbar.update(1)

        loss.backward()
        optimizer.step()
    accuracy,_,_,_,_ = test(rn, test_loader, args)
    if accuracy > best_accuracy:
        torch.save(rn.state_dict(),'./saved_model/cur_best_rn_epoch1000.pth')
        best_accuracy = accuracy
        rn.train
    if epoch%5 == 0:
        pbar.close()
        print('Accuracy of the network on the test data: %d %%' % (accuracy))
        rn.train()

EPOCH 0:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 25 %


EPOCH 5:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 56 %


EPOCH 10:   0%|          | 0/6 [00:00<?, ?it/s]

...

Accuracy of the network on the test data: 94 %


EPOCH 490:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 77 %


EPOCH 495:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 95 %


**비교하기 위한 단순 MLP 모델**


In [25]:
class MLP(nn.Module): 
    def __init__(self, args):
        super(MLP, self).__init__()
        self.latents = args.latents
        self.actions = args.actions
        self.seq_len = args.seq_len
        self.device = args.device
        self.class_num = args.class_num
        self.quantile_num = len(args.quantile)
        self.f1 = nn.Linear(self.quantile_num*(self.latents+self.actions+1), 16)
        self.f2 = nn.Linear(16, 8)
        self.f3 = nn.Linear(8, self.class_num)
        self.act1 = nn.LeakyReLU(0.01)
        self.act2 = nn.LeakyReLU(0.01)
    
    def forward(self, actions, latents, missD):
        
        bs = actions.size(0)
        actions = actions.to(args.device)
        latents = latents.to(args.device)
        missD = missD.to(args.device)


        input_concat = torch.Tensor([]).to(args.device)
        
        for i in range(self.quantile_num):
            l_q = latents[:,i,-1]
            a_q = actions[:,i,-1]
            ins = torch.cat((a_q, l_q, missD), dim = -1)
            input_concat = torch.cat((input_concat, ins), dim =-1)
        input_concat = input_concat.view(bs, -1)
        
        pred1 = self.f1(input_concat)
        pred2 = self.f2(self.act1(pred1))
        pred = self.f3(self.act2(pred2))
        
        return pred

In [26]:
mlp = MLP(args)
mlp = mlp.to(args.device)
best_accuracy2 = -10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=mlp.parameters(), lr = args.learning_rate)

In [27]:
def test2(model, test_loader, args):
    correct = [0, 0, 0, 0] # normal, autopilot lag, LOS rate delay, LOS rate bias
    total = [0, 0, 0, 0]
    model.eval()
    model = model.to(args.device)
    with torch.no_grad():
        for itr, (latent_pair, action_pair, missD_pair, index) in enumerate(test_loader):
            (_, l2) = latent_pair
            (_, a2) = action_pair
            (_, missD2) = missD_pair

            #for GPU processing
            l2 = l2.to(args.device)
            a2 = a2.to(args.device)
            missD2 = missD2.to(args.device)
            index = index.to(args.device)

            pred = mlp(a2, l2, missD2) 

            _, predicted = torch.max(pred, 1)
            #total += index.size(0)
            #correct += (predicted == index.long().squeeze()).sum().item()

            for i in range(len(index)):
                index_actual = int(index[i])
                index_predicted = predicted[i]
                total[index_actual] +=1
                if index_actual == index_predicted:
                    correct[index_actual] +=1
            
        accuracy = [0,0,0,0]
        for i in range(args.class_num):
            accuracy[i] = 100 *correct[i]/total[i]
        total_accuracy = 100* sum(correct) /sum(total)

    return total_accuracy, accuracy[0], accuracy[1], accuracy[2], accuracy[3]

In [28]:
mlp.train()

for epoch in range(args.batch_size): 
    losses = [] 
    if epoch%5 == 0:
        pbar = tqdm(total=len(train_loader), desc = "EPOCH {}".format(epoch))
    for itr, (latent_pair, action_pair, missD_pair, index) in enumerate(train_loader):
        (_, l2) = latent_pair
        (_, a2) = action_pair
        (_, missD2) = missD_pair

        #for GPU processing
        l2 = l2.to(args.device)
        a2 = a2.to(args.device)
        missD2 = missD2.to(args.device)
        index = index.to(args.device)

        pred = mlp(a2, l2, missD2) 
        optimizer.zero_grad()
        loss = criterion(pred, index.squeeze().long())
        losses.append(loss.item())
        
        if epoch%5 ==0:
            pbar.set_postfix_str("Loss={curr_loss:10.6f}, Average loss={avg_loss:10.6f}".format(curr_loss=loss.item(),avg_loss=sum(losses)/len(losses)))
            pbar.update(1)
             
        loss.backward()
        optimizer.step()
    accuracy,_,_,_,_ = test2(rn,test_loader,args)
    if accuracy > best_accuracy2:
        torch.save(mlp.state_dict(),'./saved_model/cur_best_mlp_epoch1000.pth')
        best_accuracy2 = accuracy
    if epoch%5 == 0:
        pbar.close()
        print('Accuracy of the network on the test data: %d %%' % (accuracy))
        mlp.train()

EPOCH 0:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 32 %


EPOCH 5:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 32 %


EPOCH 10:   0%|          | 0/6 [00:00<?, ?it/s]

...

Accuracy of the network on the test data: 81 %


EPOCH 490:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 81 %


EPOCH 495:   0%|          | 0/6 [00:00<?, ?it/s]

Accuracy of the network on the test data: 81 %


In [29]:
best_accuracy2

82.03125

In [21]:
# 각각 epoch에 해당하는 accuracy 그래프 그리기
# relational learning 이용 시 -> 85.15625
# 일반 mlp 이용 시 ->74.21875
# 로 10.9375 % 가량 상승 효과가 있었다.


In [30]:
rn = RN(args)
lstm = LSTM(args)
mlp = MLP(args)
lstm.eval()
rn.eval()
mlp.eval()
lstm = lstm.to(args.device)
rn = rn.to(args.device)
mlp = mlp.to(args.device)
device = 'cuda'
args.device = 'cuda'
rn.load_state_dict(torch.load('./saved_model/cur_best_rn_epoch1000.pth'))
lstm.load_state_dict(torch.load('./saved_model/cur_lstm.pth'))
mlp.load_state_dict(torch.load('./saved_model/cur_best_mlp_epoch1000.pth'))

<All keys matched successfully>

In [37]:
accuracy, a0, a1, a2, a3 = test(rn, test_loader, args)
print('Total accuracy =',accuracy)
print('Normal PNG = ', a0)
print('Autopilot Lag =', a1)
print('LOS rate delay = ', a2)
print('LOS rate bias = ', a3)

Total accuracy = 96.09375
Normal PNG =  100.0
Autopilot Lag = 100.0
LOS rate delay =  100.0
LOS rate bias =  84.375


In [38]:
accuracy2, a00, a11 ,a22, a33 = test2(mlp, test_loader, args)
print('Total accuracy =',accuracy2)
print('Normal PNG = ', a00)
print('Autopilot Lag =', a11)
print('LOS rate delay = ', a22)
print('LOS rate bias = ', a33)

Total accuracy = 82.03125
Normal PNG =  87.5
Autopilot Lag = 100.0
LOS rate delay =  100.0
LOS rate bias =  40.625
